### Tutorial on Space-time FEM with FEniCSx
Dominik Kern ORCID [0000-0002-1958-2982](https://orcid.org/0000-0002-1958-2982) 

This notebook is a supplement to the tutorial with doi [10.5281/zenodo.16761462](https://zenodo.org/records/16761462)

**solving non-dimensional wave equation in 1D with space-time FEM as BVP, i.e. with initial and final condition (not an IVP anymore)** 

In [ ]:
from mpi4py import MPI
from dolfinx import mesh, geometry
import numpy as np
import pyvista as pv
import ufl
from dolfinx.fem import functionspace
from dolfinx import default_scalar_type
from dolfinx import fem
from dolfinx.fem.petsc import LinearProblem
from dolfinx import plot

#### parameters

In [ ]:
nx = 4  # Number of spatial elements
nt = 8  # Number of time elements
order = 1  # Polynomial order

#### discretization

In [ ]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, nx, nt, mesh.CellType.quadrilateral)
V = functionspace(domain, ("Lagrange", order))

uD = fem.Function(V)   # exact solution
uD.interpolate(lambda x: np.sin(np.pi*x[0])*np.cos(np.pi*x[1]))

# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

u = ufl.TrialFunction(V)
Du = ufl.TestFunction(V)

f = fem.Constant(domain, default_scalar_type(0.0))
K = fem.Constant(domain, np.array([[1, 0], [0, -1]], dtype=np.float64))

a = ufl.dot(K*ufl.grad(u), ufl.grad(Du)) * ufl.dx   # vector equation
L = f * Du * ufl.dx

##### solution

In [ ]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

#### post-processing

In [ ]:
plotter = pv.Plotter()
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_warped = u_grid.warp_by_scalar()
plotter.add_mesh(u_warped, show_edges=True, scalar_bar_args={'vertical':True})
if not pv.OFF_SCREEN:
    plotter.show_grid(xlabel="x", ylabel="t", zlabel="u")
    plotter.show()